In [48]:
import time
import random

p1 = 4182173
p2 = 1620379
p3 = 7

p = 3
F = GF(p)
R1.<x> = PolynomialRing(F)

q = F.characteristic()

f1 = x^17+23192*x^16+3018872*x^15+1044039*x^14+216925*x^13+4052644*x^12+750793*x^11+1735730*x^10+2192751*x^9+\
          837630*x^8+3052662*x^7+2123661*x^6+40543*x^5+157093*x^4+1460955*x^3+2889718*x^2+3954822*x+3816436

f2 = x^18+1620298*x^17+3027*x^16+1551102*x^16+1087112*x^14+551994*x^13+1428730*x^12+1201892*x^11+1612689*x^12+\
          1201892*x^11+1612689*x^10+209911*x^9+91953*x^8+364830*x^7+1248613*x^6+1477676*x^5+879324*x^4+543639*x^3+\
          283719*x^2+387527*x+707074

f3 = x^5 + x^4 + x + 2

f4 = x^28 + 3*x^14 + x^7 + 3*x^21
f5 = x^9 + x^6 + 1


In [4]:
def czodd(g, d):
    if g.degree() == d:
        return [g]
    Rg = R1.quotient_by_principal_ideal(g)
    while True:
        T = R1.random_element((1, 2*d-1))
        if T.degree() < 1:
            continue
        T = Rg(T.monic())
        U = gcd(g, lift(T^((p^d-1)//2))-1)
        if U.degree() > 0 and U.degree() < g.degree():
            return czodd(U, d) + czodd(g//U, d)
        

def cz2(g, d):
    if g.degree() == d:
        return [g]
    while True:
       # T = R1.random_element((1, 2*d-1))
        T = random.randint(0, p)
        if T.degree() < 1:
            continue
        UT = T
        for i in range(d-1):
            T = T^2 % g
            UT = UT + T
        U = gcd(g, UT)
        if U.degree() > 0 and U.degree() < g.degree():
            return cz2(U, d) + cz2(g//U, d)

In [53]:
def my_factor(f, cz):
    try:
        x = f.variables()[0]
    except:
        print f
    P = x**p
    
    for d in range(1, f.degree()+1):
        print P-x
        bi = gcd(f, P-x)
        if bi != 1:
            print cz(bi, d)

        f = f // bi
        P = pow(P, p, f)

In [52]:
print 'p= ' + str(p)
print 'Source= ' + str(f5)
print ''

def start(f):
    if p == 2:
        cz = cz2
    elif p % 2 == 1:
        cz = czodd
        
    
    fd = f.derivative()
    if fd == 0:
        g = 0
        for i in range(f.degree()+1):
            if i == 0:
                g += f[i]
            elif i % q == 0 and i != 0:
                g += f[i] * x^(i/q)
        f = g  
    else:
        g = gcd(f, fd)
        if g != 0:
            f = f // g

    print f
        
    print 'my_factor:'
    %time my_factor(f, cz)

    print ''
    print 'sagemath:'
    %time print factor(f)

start(f5)



p= 3
Source= x^9 + x^6 + 1

x^3 + x^2 + 1
my_factor:
[x + 2]
[x^2 + 2*x + 2]
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 405 µs

sagemath:
(x + 2) * (x^2 + 2*x + 2)
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 557 µs


In [54]:
def solve1(f):
    x = f.variables()[0]

    g = gcd(f, x^p - x)
    
    print f
    print p

    if g.degree() == 0:
        print 'no roots'
    elif g.degree() == 1:
        print 'root: ' + str(-g[0])
    elif g.degree() == p:
        print 'roots: \all in F_p'
    else:  
        sub_solve(f, g)
    

def sub_solve(f, g):
    x = f.variables()[0]
    deg_g = g.degree()
    
    d = 1
    while d == 1 or d == g:
        delta = R1.random_element()
        temp = pow(x+delta,(p-1)/2)
        d = gcd(temp - 1, g)
        if deg_g == 1 and d == g:
            print 'root: ' + str(-d[0])
            return
    
    deg_d = d.degree()
        
    if deg_d == 1:
        print 'root: ' + str(-d[0])
        sub_solve(f, g//d)
        
    elif deg_d == deg_g - 1:
        print 'root: ' + str(-(g//d)[0])
        sub_solve(f, d)
        
    elif 2 <= deg_d < deg_g - 1:
        sub_solve(f, d)
        sub_solve(f, g//d)

In [55]:
p = 7


print gcd(f, f.derivative())
print 'f= ' + str(f4)
print ''

print 'my solve:'
%time solve1(f4)

print ''
print 'sagemath:'
%time print f4.roots()


NameError: name 'f' is not defined